In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
# Una vez ya tenemos todos nuestros datoa scrapeados y descargados. Tenemos que unir table y limpiar nuestros datos.
# Comenzamos leyendo los archivos

leagues = pd.read_csv(f'data/kaggle/leagues.csv')
players = pd.read_csv(f'data/kaggle/players.csv')
games = pd.read_csv(f'data/kaggle/games.csv')
appearances = pd.read_csv(f'data/kaggle/appearances.csv')
shots = pd.read_csv(f'data/kaggle/shots.csv')
team = pd.read_csv(f'data/kaggle/teams.csv')
team_stats = pd.read_csv(f'data/kaggle/teamstats.csv')
elo = pd.read_csv(f'data/scrapped/Elo_ranking.csv')

In [3]:
team_stats.head()

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result
0,81,89,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W
1,81,82,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L
2,82,73,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L
3,82,71,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W
4,83,72,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D


In [4]:
# La tabla que tiene los datos que necesitamos es la de team_stats, pero vemos que no tiene las ligas y necesitamos filtrar
# por la Premier League. Para ello vamos a realizar un merge primero con el nombre del equipo

data = team_stats.copy()

# Realizamos un left join para obtener los nombres de los equipos
data = pd.merge(left=data, right=team, on='teamID', how='left')
data.head()

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,name
0,81,89,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W,Manchester United
1,81,82,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L,Tottenham
2,82,73,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L,Bournemouth
3,82,71,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W,Aston Villa
4,83,72,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D,Everton


In [5]:
# Vamos a ver si tiene nulos
data.isnull().sum()

gameID           0
teamID           0
season           0
date             0
location         0
goals            0
xGoals           0
shots            0
shotsOnTarget    0
deep             0
ppda             0
fouls            0
corners          0
yellowCards      1
redCards         0
result           0
name             0
dtype: int64

In [6]:
# Tenemos un dato nulo, vamos a ver de que registro se trata. Vemos que es un partido de la Serie A. Como el proyecto se
# basa en los partidos de la Premier League no lo tomaremos en cuenta asi que no hace falta tratar este dato.

data[data.isnull().any(axis=1)]

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,name
8280,4888,95,2014,2015-03-02 19:45:00,h,1,0.355629,8,3,2,8.1923,11,1,NaN,1,D,Roma


In [7]:
# Veamos nuestra tabla de games
games.head()

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,awayProbability,homeGoalsHalfTime,awayGoalsHalfTime,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.2843,0.3999,0.3158,1,0,1.65,4.0,6.00,1.65,4.0,5.50,1.65,3.6,5.10,1.65,4.09,5.90,1.62,3.6,6.00,1.67,4.00,5.75,1.64,4.07,6.04
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.3574,0.3500,0.2926,0,0,2.00,3.6,4.00,2.00,3.3,3.70,2.10,3.3,3.30,1.95,3.65,4.27,1.91,3.5,4.00,2.00,3.50,4.20,1.82,3.88,4.70
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.2988,0.4337,0.2675,0,1,1.70,3.9,5.50,1.70,3.5,5.00,1.70,3.6,4.70,1.70,3.95,5.62,1.73,3.5,5.00,1.73,3.90,5.40,1.75,3.76,5.44
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.6422,0.2057,0.1521,3,0,1.95,3.5,4.33,2.00,3.3,3.75,2.00,3.3,3.60,1.99,3.48,4.34,2.00,3.1,2.70,2.00,3.40,4.33,1.79,3.74,5.10
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.1461,0.2159,0.6380,0,1,2.55,3.3,3.00,2.60,3.2,2.70,2.40,3.2,2.85,2.52,3.35,3.08,2.60,3.1,2.88,2.60,3.25,3.00,2.46,3.39,3.14


In [8]:
# Vamos a tener todos los datos que estamos buscando en una sola tabla para no tener que estar bucando en distintos lados

data = pd.merge(left=data, right=games, on='gameID', how='left')

In [9]:
data = pd.merge(left=data, right=leagues, on='leagueID', how='left')

In [10]:
data.columns

Index(['gameID', 'teamID', 'season_x', 'date_x', 'location', 'goals', 'xGoals',
       'shots', 'shotsOnTarget', 'deep', 'ppda', 'fouls', 'corners',
       'yellowCards', 'redCards', 'result', 'name_x', 'leagueID', 'season_y',
       'date_y', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals',
       'homeProbability', 'drawProbability', 'awayProbability',
       'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A',
       'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH',
       'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA', 'name_y',
       'understatNotation'],
      dtype='object')

In [11]:
# Ya tenemos nuestra tabla. Vamos a dropear todas aquellas columnas que no nosr siven
data['gameid'] = data['gameID']
data.drop(columns=[ 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
                   'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH',
                   'PSCD', 'PSCA', 'understatNotation', 'season_y', 'date_y', 'gameID', 'teamID', 'leagueID',
                   'homeTeamID', 'awayTeamID'], inplace=True)

In [12]:
data.rename(columns={'season_x': 'season', 'date_x': 'date', 'name_x': 'club_name', 'name_y': 'league'}, inplace=True)

In [13]:
# Obtenemos todos los partidos de la Premier League
data_pl = data[data['league']=='Premier League']
data_pl.reset_index(drop='index').to_csv(r'data/pl.csv', index=False)

In [14]:
# Vamos a entudiar solo el caso del 2015 para entender como tenemos que crear nuestra tabla y estudiar los datos.

season2015 = data_pl[data_pl['season']==2015]
season2015.head()

,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,club_name,homeGoals,awayGoals,homeProbability,drawProbability,awayProbability,league,gameid
0,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W,Manchester United,1,0,0.2843,0.3999,0.3158,Premier League,81
1,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L,Tottenham,1,0,0.2843,0.3999,0.3158,Premier League,81
2,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L,Bournemouth,0,1,0.3574,0.3500,0.2926,Premier League,82
3,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W,Aston Villa,0,1,0.3574,0.3500,0.2926,Premier League,82
4,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D,Everton,2,2,0.2988,0.4337,0.2675,Premier League,83


In [15]:
season2015.columns

Index(['season', 'date', 'location', 'goals', 'xGoals', 'shots',
       'shotsOnTarget', 'deep', 'ppda', 'fouls', 'corners', 'yellowCards',
       'redCards', 'result', 'club_name', 'homeGoals', 'awayGoals',
       'homeProbability', 'drawProbability', 'awayProbability', 'league',
       'gameid'],
      dtype='object')

In [16]:
# Vamos a unir esta tabla con los datos del fifa
fifa_2015 = pd.read_csv(r'data/scrapped/fifa_teams_16.csv')
fifa_2015.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
fifa_2015.rename(columns={'name': 'club_name'}, inplace=True)
fifa_2015

,club_name,transfer_budget,overall
0,Chelsea,85000000.0,83
1,Manchester City,100000000.0,82
2,Arsenal,70000000.0,82
3,Manchester United,80000000.0,81
4,Tottenham,45000000.0,79
5,Liverpool,55000000.0,79
6,Everton,25000000.0,78
7,Stoke,30000000.0,77
8,West Ham,30000000.0,77
9,Southampton,32000000.0,77


In [17]:
season2015 = pd.merge(left=season2015, right=fifa_2015, how='left', on='club_name')
season2015.head()

,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,club_name,homeGoals,awayGoals,homeProbability,drawProbability,awayProbability,league,gameid,transfer_budget,overall
0,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W,Manchester United,1,0,0.2843,0.3999,0.3158,Premier League,81,80000000.0,81
1,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L,Tottenham,1,0,0.2843,0.3999,0.3158,Premier League,81,45000000.0,79
2,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L,Bournemouth,0,1,0.3574,0.3500,0.2926,Premier League,82,16000000.0,73
3,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W,Aston Villa,0,1,0.3574,0.3500,0.2926,Premier League,82,29000000.0,75
4,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D,Everton,2,2,0.2988,0.4337,0.2675,Premier League,83,25000000.0,78


## Construccion de la Tabla

Vamos a contruir nuestra tabla con los datos deseados. Para ello se realizara el proceso por etapas pequeñas añadiendo cada una de las columnas por separa. Este es un proceso complejos ya que debemos extraer informacion de tablas diferentes que no tienen un comportamiento simetrico. Por lo que la logica detras de este proceso es compleja y requiere un entendimiento completo de los datos en nuestras tablas.

In [18]:
# Vamos a construir una tabla donde cada registro es un partido y las columnas representan las caracteristicas para
# los equipos locales y visitantes

columns = ['season', 'date', 'HT', 'AT', 'result', 'H_power', 'A_power', 'H_points', 'A_points', 'HT_L5GW', 'HT_L5GL',
          'HT_L5GD', 'AT_L5GW', 'AT_L5GL', 'AT_L5GD', 'HT_G', 'HT_GC', 'AT_G', 'AT_GC', 'HT_XG', 'AW_XG', 'HT_S', 'HT_SOT',
          'AT_S', 'AT_SOT', 'HT_PPDA', 'AT_PPDA', 'HT_C', 'AT_C', 'HT_Y', 'AT_Y', 'HT_R', 'AT_R']

club_name = data_pl['club_name'].unique()

df = pd.DataFrame(0, columns=columns, index=range(int(len(season2015)/2)))
for i in range(int(len(season2015)/2)):
    df.iloc[i, 0] = season2015.iloc[2*i, 0] # Temporada
    df.iloc[i, 1] = season2015.iloc[2*i, 1] 
    df.iloc[i, 2] = season2015.iloc[2*i, 14] # Nombre de equipo local
    df.iloc[i, 3] = season2015.iloc[2*i+1, 14] # Nombre de equipo visitante
        
    # Comprobamos quien gano: 1 Gano local, 2 Gano visitante, 0 Empate
    if season2015.loc[2*i, 'result'] == 'W':
        df.iloc[i, 4] = 1
    elif season2015.loc[2*i, 'result'] == 'L':
        df.iloc[i, 4] = 2
    else:
        df.iloc[i, 4] = 0

#     Vamos a añadir los goles esperados para cada equipo
    df.iloc[i, 19] = season2015.iloc[2*i, 4]
    df.iloc[i, 20] = season2015.iloc[2*i+1, 4]

# Vamos a obtener los puntos acumulados hasta el partido para cada uno de los registros
for i in range(int(len(season2015)/2)):
    for club in club_name:        
        small = df[:i]
        if season2015.iloc[2*i, 14] == club:
            ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
            ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']
            
            df.iloc[i, 7] = ptw + ptd # Puntos acumulados equipo local
            
            L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
            # Contamos las victorias, derrotas y empates (ultimos 5) del equipo local
            df.iloc[i, 9] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
            df.iloc[i, 10] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
            df.iloc[i, 11] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']
            
            df.iloc[i, 5] = season2015.iloc[2*i, -1] # Poder del equipo local
        
        if season2015.iloc[2*i+1, 14] == club:
            ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
            ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']
            df.iloc[i, 8] = ptw + ptd # Puntos acumulados equipo visitante
            
            L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
            # Contamos las victorias, derrotas y empates (ultimos 5) del equipo visitante
            df.iloc[i, 12] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
            df.iloc[i, 13] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
            df.iloc[i, 14] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']
            
            df.iloc[i, 6] = season2015.iloc[2*i+1, -1] # Poder del equipo visitante

In [19]:
# Vamos a colocar el resto de valores en nuestro DataFrame
for i in range(int(len(season2015)/2)):
    for club in club_name:        
        
        small = season2015[:2*i+1][:-1]
        if season2015.iloc[2*i, 14] == club:
            ind = small[small['club_name'] == club]['gameid'] # Obtenemos los ID de los partidos
            df.iloc[i, 15] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados HT
            df.iloc[i, 16] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos HT

        if season2015.iloc[2*i+1, 14] == club: # Loop equipo visitante
            ind = small[small['club_name'] == club]['gameid'] 
            df.iloc[i, 17] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados AT
            df.iloc[i, 18] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos AT

In [21]:
# Para el resto de los datos solo se tomaran en cuenta los ultimos 5 partidos
for i in range(int(len(season2015)/2)):
    for club in club_name:        
        
        small = season2015[:2*i+1][:-1]
        if season2015.iloc[2*i, 14] == club:
            
            # Tiros y tiros al arco realizados en los ultimos 5 partidos para el HT
            df.iloc[i, 21] = small[small['club_name'] == club][-5:]['shots'].values.sum()
            df.iloc[i, 22] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()
            
            # Media PPDA HT ultimos 5 partidos
            df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
            
            # Corners HT ultimos 5 partidos
            df.iloc[i, 27] = small[small['club_name'] == club][-5:]['corners'].values.sum()
            
            # Tarjetas amarillas ultimos 5 partidos HT
            df.iloc[i, 29] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()
            
            # Tarjetas rojas ultimos 5 partidos HT
            df.iloc[i, 31] = small[small['club_name'] == club][-5:]['redCards'].values.sum()
            
        

        if season2015.iloc[2*i+1, 14] == club: # Loop equipo visitante
            # Tiros y tiros al arco realizados en los ultimos 5 partidos para el AT
            df.iloc[i, 23] = small[small['club_name'] == club][-5:]['shots'].values.sum()
            df.iloc[i, 24] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()
            
            # Media PPDA AT ultimos 5 partidos
            df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
            
            # Corners AT ultimos 5 partidos
            df.iloc[i, 28] = small[small['club_name'] == club][-5:]['corners'].values.sum()
            
            # Tarjetas amarillas ultimos 5 partidos AT
            df.iloc[i, 30] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()
            
            # Tarjetas rojas ultimos 5 partidos AT
            df.iloc[i, 32] = small[small['club_name'] == club][-5:]['redCards'].values.sum()

<ipython-input-21-85dbbf269fbb>:13: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
C:\Users\sasim\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-21-85dbbf269fbb>:32: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()


In [22]:
df

,season,date,HT,AT,result,H_power,A_power,H_points,A_points,HT_L5GW,HT_L5GL,HT_L5GD,AT_L5GW,AT_L5GL,AT_L5GD,HT_G,HT_GC,AT_G,AT_GC,HT_XG,AW_XG,HT_S,HT_SOT,AT_S,AT_SOT,HT_PPDA,AT_PPDA,HT_C,AT_C,HT_Y,AT_Y,HT_R,AT_R
0,2015,2015-08-08 15:45:00,Manchester United,Tottenham,1,81,79,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.627539,0.674600,0,0,0,0,NaN,NaN,0,0,0.0,0.0,0,0
1,2015,2015-08-08 18:00:00,Bournemouth,Aston Villa,2,73,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.876106,0.782253,0,0,0,0,NaN,NaN,0,0,0.0,0.0,0,0
2,2015,2015-08-08 18:00:00,Everton,Watford,0,78,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.604226,0.557892,0,0,0,0,NaN,NaN,0,0,0.0,0.0,0,0
3,2015,2015-08-08 18:00:00,Leicester,Sunderland,1,76,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.568030,1.459460,0,0,0,0,NaN,NaN,0,0,0.0,0.0,0,0
4,2015,2015-08-08 18:00:00,Norwich,Crystal Palace,2,74,76,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.130760,2.109750,0,0,0,0,NaN,NaN,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2015,2016-05-15 18:00:00,Stoke,West Ham,1,77,77,48,62,0,4,1,3,1,1,39.0,54.0,64.0,49.0,0.288737,2.944890,60,13,90,28,9.57018,7.09960,17,34,9.0,7.0,0,0
376,2015,2016-05-15 18:00:00,Swansea,Manchester City,0,76,82,46,65,3,2,0,2,1,2,41.0,51.0,70.0,40.0,0.115604,2.040030,63,23,51,27,8.78666,8.14264,21,24,10.0,8.0,0,0
377,2015,2016-05-15 18:00:00,Watford,Sunderland,0,75,75,44,38,2,3,0,3,0,2,38.0,48.0,46.0,60.0,1.952770,0.908817,62,24,62,23,8.38512,12.58166,22,23,12.0,11.0,1,0
378,2015,2016-05-15 18:00:00,West Bromwich Albion,Liverpool,0,75,79,42,59,0,3,2,2,1,2,33.0,47.0,62.0,49.0,1.868690,0.419877,68,17,108,34,11.38740,7.15974,33,36,3.0,8.0,0,1


Ahora se procedera a unir todas las celdas de codigo anteriores en una unica celda para observar si el codigo se comporta bien.

In [24]:
# Ahora que entendimos como construir la tabla anterior, tenemos que contruir la misma tabla pero para cada partido desde
# la temporada 2015-2016 hasta la 2020-2021. Los partidos al comienzo de la temporada no tendran datos

columns = ['season', 'date', 'HT', 'AT', 'result', 'H_power', 'A_power', 'H_points', 'A_points', 'HT_L5GW', 'HT_L5GL',
          'HT_L5GD', 'AT_L5GW', 'AT_L5GL', 'AT_L5GD', 'HT_G', 'HT_GC', 'AT_G', 'AT_GC', 'HT_XG', 'AW_XG', 'HT_S', 'HT_SOT',
          'AT_S', 'AT_SOT', 'HT_PPDA', 'AT_PPDA', 'HT_C', 'AT_C', 'HT_Y', 'AT_Y', 'HT_R', 'AT_R']

club_name = data_pl['club_name'].unique()
season = data_pl[data_pl['season']==2016]
season.reset_index(drop=True, inplace=True)

fifa = pd.read_csv(r'data/scrapped/fifa_teams_17.csv')
fifa.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
fifa.rename(columns={'name': 'club_name'}, inplace=True)

season = pd.merge(left=season, right=fifa, how='left', on='club_name')

df = pd.DataFrame(0, columns=columns, index=range(int(len(season)/2)))
for i in range(int(len(season)/2)):
    df.iloc[i, 0] = season.iloc[2*i, 0] # Temporada
    df.iloc[i, 1] = season.iloc[2*i, 1] 
    df.iloc[i, 2] = season.iloc[2*i, 14] # Nombre de equipo local
    df.iloc[i, 3] = season.iloc[2*i+1, 14] # Nombre de equipo visitante
        
    # Comprobamos quien gano: 1 Gano local, 2 Gano visitante, 0 Empate
    if season.loc[2*i, 'result'] == 'W':
        df.iloc[i, 4] = 1
    elif season.loc[2*i, 'result'] == 'L':
        df.iloc[i, 4] = 2
    else:
        df.iloc[i, 4] = 0

    # Vamos a añadir los goles esperados para cada equipo
    df.iloc[i, 19] = season.iloc[2*i, 4]
    df.iloc[i, 20] = season.iloc[2*i+1, 4]

    
# Vamos a obtener los puntos acumulados hasta el partido para cada uno de los registros
for i in range(int(len(season)/2)):
    for club in club_name:        
        small = df[:i]
        if season.iloc[2*i, 14] == club:
            ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
            ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']
            
            df.iloc[i, 7] = ptw + ptd # Puntos acumulados equipo local
            
            L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
            # Contamos las victorias, derrotas y empates (ultimos 5) del equipo local
            df.iloc[i, 9] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
            df.iloc[i, 10] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
            df.iloc[i, 11] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']
            
            df.iloc[i, 5] = season.iloc[2*i, -1] # Poder del equipo local
        
        if season.iloc[2*i+1, 14] == club:
            ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
            ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']
            df.iloc[i, 8] = ptw + ptd # Puntos acumulados equipo visitante
            
            L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
            # Contamos las victorias, derrotas y empates (ultimos 5) del equipo visitante
            df.iloc[i, 12] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
            df.iloc[i, 13] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
            df.iloc[i, 14] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']
            
            df.iloc[i, 6] = season.iloc[2*i+1, -1] # Poder del equipo visitante
            
                 
for i in range(int(len(season)/2)):
    for club in club_name:        
        
        small = season[:2*i+1][:-1]
        if season.iloc[2*i, 14] == club:
            ind = small[small['club_name'] == club]['gameid'] # Obtenemos los ID de los partidos
            df.iloc[i, 15] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados HT
            df.iloc[i, 16] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos HT

        if season.iloc[2*i+1, 14] == club: # Loop equipo visitante
            ind = small[small['club_name'] == club]['gameid'] 
            df.iloc[i, 17] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados AT
            df.iloc[i, 18] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos AT
            
            
            
for i in range(int(len(season)/2)):
    for club in club_name:        
        
        small = season[:2*i+1][:-1]
        if season.iloc[2*i, 14] == club:
            
            # Tiros y tiros al arco realizados en los ultimos 5 partidos para el HT
            df.iloc[i, 21] = small[small['club_name'] == club][-5:]['shots'].values.sum()
            df.iloc[i, 22] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()
            
            # Media PPDA HT ultimos 5 partidos
            df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
            
            # Corners HT ultimos 5 partidos
            df.iloc[i, 27] = small[small['club_name'] == club][-5:]['corners'].values.sum()
            
            # Tarjetas amarillas ultimos 5 partidos HT
            df.iloc[i, 29] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()
            
            # Tarjetas rojas ultimos 5 partidos HT
            df.iloc[i, 31] = small[small['club_name'] == club][-5:]['redCards'].values.sum()
            
        

        if season2015.iloc[2*i+1, 14] == club: # Loop equipo visitante
            # Tiros y tiros al arco realizados en los ultimos 5 partidos para el AT
            df.iloc[i, 23] = small[small['club_name'] == club][-5:]['shots'].values.sum()
            df.iloc[i, 24] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()
            
            # Media PPDA AT ultimos 5 partidos
            df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
            
            # Corners AT ultimos 5 partidos
            df.iloc[i, 28] = small[small['club_name'] == club][-5:]['corners'].values.sum()
            
            # Tarjetas amarillas ultimos 5 partidos AT
            df.iloc[i, 30] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()
            
            # Tarjetas rojas ultimos 5 partidos AT
            df.iloc[i, 32] = small[small['club_name'] == club][-5:]['redCards'].values.sum()

In [25]:
df

,season,date,HT,AT,result,H_power,A_power,H_points,A_points,HT_L5GW,HT_L5GL,HT_L5GD,AT_L5GW,AT_L5GL,AT_L5GD,HT_G,HT_GC,AT_G,AT_GC,HT_XG,AW_XG,HT_S,HT_SOT,AT_S,AT_SOT,HT_PPDA,AT_PPDA,HT_C,AT_C,HT_Y,AT_Y,HT_R,AT_R
0,2016,2016-08-13 15:30:00,Hull,Leicester,1,75,78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.740018,2.456310,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
1,2016,2016-08-13 18:00:00,Crystal Palace,West Bromwich Albion,2,77,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.990062,0.684593,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
2,2016,2016-08-13 18:00:00,Everton,Tottenham,0,80,81,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.790063,1.123480,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
3,2016,2016-08-13 18:00:00,Burnley,Swansea,2,74,77,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.049030,1.720890,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
4,2016,2016-08-13 18:00:00,Middlesbrough,Stoke,0,75,78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.805230,0.485626,0,0,14,4,0.00000,0.85218,0,3,0.0,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2016,2017-05-21 15:00:00,Liverpool,Middlesbrough,1,81,75,73,28,3,1,1,0,4,1,75.0,42.0,27.0,50.0,2.169570,0.899691,84,30,64,19,7.05766,15.17616,22,23,4.0,10.0,0,0
376,2016,2017-05-21 15:00:00,Manchester United,Crystal Palace,1,83,77,66,41,2,2,1,2,2,1,52.0,29.0,50.0,61.0,1.038600,0.301600,51,21,104,36,12.71890,7.20220,21,44,8.0,9.0,0,0
377,2016,2017-05-21 15:00:00,Southampton,Stoke,2,78,78,46,41,1,2,2,1,2,2,41.0,47.0,40.0,56.0,1.355400,0.983464,49,11,55,19,11.48722,14.18450,25,22,8.0,11.0,0,0
378,2016,2017-05-21 15:00:00,Swansea,West Bromwich Albion,1,77,75,38,45,3,1,1,0,4,1,43.0,69.0,42.0,49.0,1.745550,1.606740,47,21,94,38,16.32932,6.86350,28,21,4.0,3.0,0,0


## Tabla con todas las temporadas

Vamos a realizar un loop en donde se haga la tabla anterior pero para todas las temporadas en nuestro dataset. Luego todas las tablas se uniran en un unico archivo y se exportara a un .csv para su posterior uso.

In [23]:
# Ahora que entendimos como construir la tabla anterior, tenemos que contruir la misma tabla pero para cada partido desde
# la temporada 2015-2016 hasta la 2020-2021. Los partidos al comienzo de la temporada no tendran datos

columns = ['season', 'date', 'HT', 'AT', 'result', 'H_power', 'A_power', 'H_points', 'A_points', 'HT_L5GW', 'HT_L5GL',
          'HT_L5GD', 'AT_L5GW', 'AT_L5GL', 'AT_L5GD', 'HT_G', 'HT_GC', 'AT_G', 'AT_GC', 'HT_XG', 'AW_XG', 'HT_S', 'HT_SOT',
          'AT_S', 'AT_SOT', 'HT_PPDA', 'AT_PPDA', 'HT_C', 'AT_C', 'HT_Y', 'AT_Y', 'HT_R', 'AT_R']

club_name = data_pl['club_name'].unique()
temp = [2015, 2016, 2017, 2018, 2019, 2020]
fifa_year = ['16', '17', '18', '19', '20', '21']

df_final = pd.DataFrame(columns=columns)
for j in range(len(temp)):
    season = data_pl[data_pl['season']==temp[j]]
    season.reset_index(drop=True, inplace=True)

    fifa = pd.read_csv(f'data\\scrapped\\fifa_teams_{fifa_year[j]}.csv')
    fifa.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
    fifa.rename(columns={'name': 'club_name'}, inplace=True)

    season = pd.merge(left=season, right=fifa, how='left', on='club_name')

    df = pd.DataFrame(0, columns=columns, index=range(int(len(season)/2)))
    for i in range(int(len(season)/2)):
        df.iloc[i, 0] = season.iloc[2*i, 0] # Temporada
        df.iloc[i, 1] = season.iloc[2*i, 1] # Fecha
        df.iloc[i, 2] = season.iloc[2*i, 14] # Nombre de equipo local
        df.iloc[i, 3] = season.iloc[2*i+1, 14] # Nombre de equipo visitante

        # Comprobamos quien gano: 1 Gano local, 2 Gano visitante, 0 Empate
        if season.loc[2*i, 'result'] == 'W':
            df.iloc[i, 4] = 1
        elif season.loc[2*i, 'result'] == 'L':
            df.iloc[i, 4] = 2
        else:
            df.iloc[i, 4] = 0

        # Vamos a añadir los goles esperados para cada equipo
        df.iloc[i, 19] = season.iloc[2*i, 4]
        df.iloc[i, 20] = season.iloc[2*i+1, 4]


    # Vamos a obtener los puntos acumulados hasta el partido para cada uno de los registros
    for i in range(int(len(season)/2)):
        for club in club_name:        
            small = df[:i]
            if season.iloc[2*i, 14] == club:
                ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
                ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']

                df.iloc[i, 7] = ptw + ptd # Puntos acumulados equipo local

                L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
                # Contamos las victorias, derrotas y empates (ultimos 5) del equipo local
                df.iloc[i, 9] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
                df.iloc[i, 10] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
                df.iloc[i, 11] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']

                df.iloc[i, 5] = season.iloc[2*i, -1] # Poder del equipo local

            if season.iloc[2*i+1, 14] == club:
                ptw = small[((small['HT'] == club) & (small['result'] == 1)) | ((small['AT'] == club) & (small['result'] == 2))].count()['result'] * 3
                ptd = small[((small['HT'] == club) & (small['result'] == 0)) | ((small['AT'] == club) & (small['result'] == 0))].count()['result']
                df.iloc[i, 8] = ptw + ptd # Puntos acumulados equipo visitante

                L5M = small[(small['HT'] == club) | (small['AT'] == club)][-5:]
                # Contamos las victorias, derrotas y empates (ultimos 5) del equipo visitante
                df.iloc[i, 12] = L5M[((L5M['HT'] == club) & (L5M['result'] == 1)) | ((L5M['AT'] == club) & (L5M['result'] == 2))].count()['result']
                df.iloc[i, 13] = L5M[((L5M['HT'] == club) & (L5M['result'] == 2)) | ((L5M['AT'] == club) & (L5M['result'] == 1))].count()['result']
                df.iloc[i, 14] = L5M[((L5M['HT'] == club) & (L5M['result'] == 0)) | ((L5M['AT'] == club) & (L5M['result'] == 0))].count()['result']

                df.iloc[i, 6] = season.iloc[2*i+1, -1] # Poder del equipo visitante


    # Goles realizados y concedidos
    for i in range(int(len(season)/2)):
        for club in club_name:        

            small = season[:2*i+1][:-1]
            if season.iloc[2*i, 14] == club:
                ind = small[small['club_name'] == club]['gameid'] # Obtenemos los ID de los partidos
                df.iloc[i, 15] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados HT
                df.iloc[i, 16] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos HT

            if season.iloc[2*i+1, 14] == club: # Loop equipo visitante
                ind = small[small['club_name'] == club]['gameid'] 
                df.iloc[i, 17] = small[(small.gameid.isin(ind)) & (small['club_name'] == club)].sum()['goals'] # Goles realizados AT
                df.iloc[i, 18] = small[(small.gameid.isin(ind)) & (small['club_name'] != club)].sum()['goals'] # Goles concedidos AT



    for i in range(int(len(season)/2)):
        for club in club_name:        

            small = season[:2*i+1][:-1]
            if season.iloc[2*i, 14] == club:

                # Tiros y tiros al arco realizados en los ultimos 5 partidos para el HT
                df.iloc[i, 21] = small[small['club_name'] == club][-5:]['shots'].values.sum()
                df.iloc[i, 22] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()

                # Media PPDA HT ultimos 5 partidos
                df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()

                # Corners HT ultimos 5 partidos
                df.iloc[i, 27] = small[small['club_name'] == club][-5:]['corners'].values.sum()

                # Tarjetas amarillas ultimos 5 partidos HT
                df.iloc[i, 29] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()

                # Tarjetas rojas ultimos 5 partidos HT
                df.iloc[i, 31] = small[small['club_name'] == club][-5:]['redCards'].values.sum()


            if season.iloc[2*i+1, 14] == club: # Loop equipo visitante
                # Tiros y tiros al arco realizados en los ultimos 5 partidos para el AT
                df.iloc[i, 23] = small[small['club_name'] == club][-5:]['shots'].values.sum()
                df.iloc[i, 24] = small[small['club_name'] == club][-5:]['shotsOnTarget'].values.sum()

                # Media PPDA AT ultimos 5 partidos
                df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()

                # Corners AT ultimos 5 partidos
                df.iloc[i, 28] = small[small['club_name'] == club][-5:]['corners'].values.sum()

                # Tarjetas amarillas ultimos 5 partidos AT
                df.iloc[i, 30] = small[small['club_name'] == club][-5:]['yellowCards'].values.sum()

                # Tarjetas rojas ultimos 5 partidos AT
                df.iloc[i, 32] = small[small['club_name'] == club][-5:]['redCards'].values.sum()
                
    df_final = pd.concat([df_final, df], axis=0)

<ipython-input-23-94db3d699715>:103: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
C:\Users\sasim\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-23-94db3d699715>:121: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
<ipython-input-23-94db3d699715>:121: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 26] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
C:\Users\sasim\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-23-94db3d699715>:103: RuntimeWarning: Mean of empty slice.
  df.iloc[i, 25] = small[small['club_name'] == club][-5:]['ppda'].values.mean()
<ipython-input-23-94db3d699715>:121: RuntimeWarn

In [27]:
df_final

,season,date,HT,AT,result,H_power,A_power,H_points,A_points,HT_L5GW,HT_L5GL,HT_L5GD,AT_L5GW,AT_L5GL,AT_L5GD,HT_G,HT_GC,AT_G,AT_GC,HT_XG,AW_XG,HT_S,HT_SOT,AT_S,AT_SOT,HT_PPDA,AT_PPDA,HT_C,AT_C,HT_Y,AT_Y,HT_R,AT_R
0,2015,2015-08-08 15:45:00,Manchester United,Tottenham,1,81,79,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.627539,0.674600,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
1,2015,2015-08-08 18:00:00,Bournemouth,Aston Villa,2,73,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.876106,0.782253,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
2,2015,2015-08-08 18:00:00,Everton,Watford,0,78,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.604226,0.557892,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
3,2015,2015-08-08 18:00:00,Leicester,Sunderland,1,76,75,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.568030,1.459460,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
4,2015,2015-08-08 18:00:00,Norwich,Crystal Palace,2,74,76,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.130760,2.109750,0,0,0,0,0.00000,0.00000,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2020,2021-05-23 15:00:00,Liverpool,Crystal Palace,1,85,76,66,44,4,0,1,2,3,0,66.0,42.0,41.0,64.0,2.203480,0.772317,99,32,69,28,8.96522,15.35568,38,28,2.0,9.0,0,0
376,2020,2021-05-23 15:00:00,Manchester City,Everton,1,85,79,83,59,3,2,0,3,2,0,78.0,32.0,47.0,43.0,2.883550,1.072580,76,27,63,17,12.09556,14.42834,25,30,8.0,5.0,1,0
377,2020,2021-05-23 15:00:00,Sheffield United,Burnley,1,73,76,20,39,2,3,0,2,3,0,19.0,63.0,33.0,54.0,0.420006,0.558508,43,8,62,23,16.75322,12.06928,19,26,10.0,5.0,0,0
378,2020,2021-05-23 15:00:00,West Ham,Southampton,1,78,76,62,43,2,2,1,1,3,1,59.0,47.0,47.0,65.0,1.983440,1.351790,78,18,57,25,11.48794,16.06946,24,19,6.0,2.0,1,1


In [24]:
df_final.isnull().sum()

season       0
date         0
HT           0
AT           0
result       0
H_power      0
A_power      0
H_points     0
A_points     0
HT_L5GW      0
HT_L5GL      0
HT_L5GD      0
AT_L5GW      0
AT_L5GL      0
AT_L5GD      0
HT_G         0
HT_GC        0
AT_G         0
AT_GC        0
HT_XG        0
AW_XG        0
HT_S         0
HT_SOT       0
AT_S         0
AT_SOT       0
HT_PPDA     60
AT_PPDA     60
HT_C         0
AT_C         0
HT_Y         0
AT_Y         0
HT_R         0
AT_R         0
dtype: int64

In [27]:
# Tenemos NaN en los PPDA al inicio de la temporada. Los cambiamos por 0.
df_final.fillna(value=0, inplace=True)

In [28]:
# Por ultimo, vamos a guardar nuestros datos en un .csv para no tener que realizar este proceso multiples veces.

df_final.reset_index(drop='index').to_csv(r'data/football_statistics_pl.csv', index=False)

In [29]:
df_final.reset_index(drop=True, inplace=True)

## ELO Rating

In [30]:
# Ahora vamos a añadir a la tabla el ELO Ranking. Para mas informacion visitar:
#     https://www.eloratings.net/about

elo = pd.read_csv(f'data/scrapped/Elo_ranking.csv')
elo.drop(columns=['Unnamed: 0'], inplace=True)

In [31]:
elo # Tenemos el elo de cada uno de nuestro equipo. Este dato sera modificado tras cada partido y se vera afectado si
    # pierden ganan o empatan!

,name,ELO_ranking
0,Chelsea,1889.0
1,Manchester City,1880.0
2,Arsenal,1849.0
3,Manchester United,1808.0
4,Tottenham,1728.0
5,Liverpool,1750.0
6,Everton,1704.0
7,Stoke,1696.0
8,West Ham,1594.0
9,Southampton,1716.0


In [32]:
def elo_rating(HT_elo_old, AT_elo_old, HT_goals, AT_goals, result):
    wht = 1 / (1 + 10 ** ((AT_elo_old-HT_elo_old)/400))
    wat = 1 / (1 + 10 ** ((HT_elo_old-AT_elo_old)/400))
    goal_diff = abs(HT_goals-AT_goals)
    k = 30
    
    if goal_diff == 1 or goal_diff == 0:
        G = 1
    elif goal_diff == 2:
        G = 3/4
    else:
        G = (3/4) + (goal_diff-3) / 8
    
    if result == 1: # Gano equipo local y perdio visitante
        HT_elo_new = HT_elo_old + k * G * (1 - wht)
        AT_elo_new = AT_elo_old + k * G * (0 - wat)
        
    elif result == 2: # Gano equipo visitante y perdio el local
        HT_elo_new = HT_elo_old + k * G * (0 - wht)
        AT_elo_new = AT_elo_old + k * G * (1 - wat)
        
    else: # Empate
        HT_elo_new = HT_elo_old + k * G * (0.5 - wht)
        AT_elo_new = AT_elo_old + k * G * (0.5 - wat)
        
    return HT_elo_new, AT_elo_new

In [33]:
elo_HT = []
elo_AT = []
for i in range(len(df_final)):
    elo_HT.append(elo[elo['name']==df_final.loc[i, 'HT']]['ELO_ranking'].values[0])
    elo_AT.append(elo[elo['name']==df_final.loc[i, 'AT']]['ELO_ranking'].values[0])
    
    HT_elo_new, AT_elo_new = elo_rating(elo_HT[i], elo_AT[i], data_pl[data_pl['season']!=2014].iloc[2*i, 3], 
                                        data_pl[data_pl['season']!=2014].iloc[2*i+1, 3], df_final.loc[i, 'result'])
    
    elo.iloc[elo[elo['name']==df_final.loc[i, 'HT']]['ELO_ranking'].index, 1] = HT_elo_new
    elo.iloc[elo[elo['name']==df_final.loc[i, 'AT']]['ELO_ranking'].index, 1] = AT_elo_new

In [34]:
df_final['ELO_HT'] = elo_HT
df_final['ELO_AT'] = elo_AT

# Guardamos la nueva tabla
df_final.reset_index(drop='index').to_csv(r'data/football_statistics_pl.csv', index=False)